# Data Set generation for detection of pilot contamination in multi user Massive MIMO Systems

This notebook provides the code that generates the dataset based on SNR values and transmit power at the eavesdropper.

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig

In [2]:
def bin2gray(bi):
    order = bi.shape[1]
    lin = bi.shape[0]
    g = np.zeros((lin, order))

    idx = 0
    for bits in bi:
        g[idx, 0] = int(bits[0])
        for l in range(1, order):
            g[idx, l] = int(bits[l] ^ bits[l - 1])
        idx = idx + 1

    return g

def qpskmodulator(input_bits, average_power=1, phase_offset=np.pi / 4):

    # Generate symbols
    symbols = np.array([1, 1j, -1, -1j]) * np.exp(1j * phase_offset)
    symbols = np.sqrt(average_power) * symbols

    input_arr = input_bits.reshape((int(len(input_bits) / 2)), 2)
    gray_code = bin2gray(input_arr)

    output = [
    ]  # Pre allocate in the future to prevent unnecessary memory usage
    for symbol in gray_code:
        #         print(symbol)
        s = int(2 * symbol[0] + symbol[1])
        output.append(symbols[s])

    return np.array(output)


def awgn(sig, SNR, sig_power=1):
    # SNR in dB

    SNR_linear = 10**(SNR / 10)  #SNR to linear scale
    noise_power = sig_power / SNR_linear
    # Noise power

    dim = sig.shape
    if not np.isreal(sig[1, 1]):
        noise = np.sqrt(noise_power / 2) * (np.random.normal(0, 1, dim) +
                                            1j * np.random.normal(0, 1, dim))
    else:
        noise = np.sqrt(noise_power) * np.random.normal(0, 1, dim)

    sig_noisy = sig + noise

    return sig_noisy

## System Parameters

In [3]:
nAntennas            = 64 #qtd de antenas na estacao base
nUsers               = 4 #qtd de usuarios conectados na rede
nEves                = 1 #qtd de espioes 
nBitsPerSymbol       = 2 #QPSK
nTrials              = 1 #repetibilidade de cada iteracao
usr_power            = 1 # potencia dos usuarios
nPilotSymbols_values = np.arange(5, 105, 10) #range de qtd de simbolos na sequencia piloto que quero testar
eve_power_values     = np.arange(0, 5.1, 0.5) #range de potencia do espiao que quero testar
snr_values           = np.arange(0, 31) #range de snrs que quero testar

## Simulates MIMO Channel Fading with AWGN

In [4]:
# LAYOUT DO PRINT
status_message = 'Trial {}: nPilotSymbols={}, snr={}, eve_power={}'

for nPilotSymbols in nPilotSymbols_values:

    # PRA CADA SNR
    for snr in snr_values:

        # PRA CADA POTENCIA DO ESPIAO
        for eve_power in eve_power_values:

            # REPETIBILIDADE
            for trial in range(nTrials):

                # PRINT
                print(status_message.format(trial+1, nPilotSymbols, snr, eve_power))

                # SIMULANDO OS CANAIS DO USUARIO E DO ESPIAO
                Haut = np.sqrt(0.5)*(np.random.normal(0, 1, size=(nAntennas, nUsers)) + 1j*np.random.normal(0, 1, size=(nAntennas, nUsers)))
                g    = np.sqrt(0.5)*(np.random.normal(0, 1, size=(nAntennas, nEves)) + 1j*np.random.normal(0, 1, (nAntennas, nEves)))

                # SEQUENCIA PILOTO ALEATORIA + MODULACAO PARA TODOS OS USUARIOS:
                bitStream = np.random.choice([0, 1], nPilotSymbols*nBitsPerSymbol*nUsers)
                symb      = qpskmodulator(bitStream) # QPSK Modulator
                xp        = symb.reshape(nUsers, nPilotSymbols)

                # ESPIAO ENTRANDO NA JOGADA (SE A POTENCIA DELE FOR 0 ELE NAO ENTRA NA JOGADA):
                xpe        = np.sqrt(eve_power)*xp[0, :] #xpe vai ser a sequencia piloto do primeiro usuario multiplicada pela raiz da potencia do espiao
                xptx       = np.concatenate((xp, [xpe])) #xptx sera a matriz xp com uma linha a mais: xpe
                H          = np.concatenate((Haut, g), axis=1) #H vai ser Haut com uma COLUNA a mais, que vai ser g
                
                # TRANSMISSAO PELO CANAL
                yp = np.dot(H, xptx) # fading
                yp = awgn(yp, SNR=snr) #ruido branco

                # Flag to determine the target in the data set
                if eve_power == 0:
                    contamination = 0
                else:
                    contamination = 1

Trial 1: nPilotSymbols=5, snr=0, eve_power=0.0
Trial 1: nPilotSymbols=5, snr=0, eve_power=0.5
Trial 1: nPilotSymbols=5, snr=0, eve_power=1.0
Trial 1: nPilotSymbols=5, snr=0, eve_power=1.5
Trial 1: nPilotSymbols=5, snr=0, eve_power=2.0
Trial 1: nPilotSymbols=5, snr=0, eve_power=2.5
Trial 1: nPilotSymbols=5, snr=0, eve_power=3.0
Trial 1: nPilotSymbols=5, snr=0, eve_power=3.5
Trial 1: nPilotSymbols=5, snr=0, eve_power=4.0
Trial 1: nPilotSymbols=5, snr=0, eve_power=4.5
Trial 1: nPilotSymbols=5, snr=0, eve_power=5.0
Trial 1: nPilotSymbols=5, snr=1, eve_power=0.0
Trial 1: nPilotSymbols=5, snr=1, eve_power=0.5
Trial 1: nPilotSymbols=5, snr=1, eve_power=1.0
Trial 1: nPilotSymbols=5, snr=1, eve_power=1.5
Trial 1: nPilotSymbols=5, snr=1, eve_power=2.0
Trial 1: nPilotSymbols=5, snr=1, eve_power=2.5
Trial 1: nPilotSymbols=5, snr=1, eve_power=3.0
Trial 1: nPilotSymbols=5, snr=1, eve_power=3.5
Trial 1: nPilotSymbols=5, snr=1, eve_power=4.0
Trial 1: nPilotSymbols=5, snr=1, eve_power=4.5
Trial 1: nPil

In [8]:
yp.shape

(64, 95)